# Using Macauff Cross-match results in LSDB

This notebook goes over loading an association table that has been imported from a macauff cross-match result, using it to join lsdb catalogs, and performing queries on the joined datasets.

First, the necesarry imports and setting up a Dask Client

In [1]:
import lsdb

from dask.distributed import Client

In [2]:
client = Client(
    n_workers=6,
    threads_per_worker=1,
    memory_limit="20GB",
    local_directory="/epyc/data3/hats/tmp/macauff",
)
client

/astro/users/smcampos/.conda/envs/lsdb_v06/lib/python3.12/site-packages/distributed/node.py:187: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 36215 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:36215/status,
Dashboard: http://127.0.0.1:36215/status,Workers: 6
Total threads: 6,Total memory: 111.76 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:36589,Workers: 6
Dashboard: http://127.0.0.1:36215/status,Total threads: 6
Started: Just now,Total memory: 111.76 GiB
Comm: tcp://127.0.0.1:43997,Total threads: 1
Dashboard: http://127.0.0.1:43729/status,Memory: 18.63 GiB
Nanny: tcp://127.0.0.1:35951,


## Loading the catalogs

Each catalog is loaded using the `lsdb.open_catalog` method. First the macauff association table, and then the gaia and wise object catalogs. The loading is done lazily, so at this stage the results show just the column names and types.

In [3]:
macauff_association = lsdb.open_catalog("/epyc/data3/hats/tmp/macauff_association")
macauff_association

,gaia_source_id,gaia_ra,gaia_dec,BP,G,RP,catwise_name,catwise_ra,catwise_dec,W1,W2,match_p,separation,eta,xi,gaia_avg_cont,catwise_avg_cont,gaia_cont_f1,gaia_cont_f10,catwise_cont_f1,catwise_cont_f10,catwise_fit_sig
npartitions=3933,,,,,,,,,,,,,,,,,,,,,,
"Order: 2, Pixel: 0",int64[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],string[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow]
"Order: 3, Pixel: 4",...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"Order: 4, Pixel: 3067",...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"Order: 3, Pixel: 767",...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [4]:
gaia = lsdb.open_catalog("/epyc/data3/hats/catalogs/gaia_dr3")
gaia

,solution_id,designation,source_id,ref_epoch,ra,ra_error,dec,dec_error,parallax,parallax_error,pm,pmra,pmra_error,pmdec,pmdec_error,phot_g_n_obs,phot_g_mean_flux,phot_g_mean_flux_error,phot_g_mean_mag,phot_bp_n_obs,phot_bp_mean_flux,phot_bp_mean_flux_error,phot_bp_mean_mag,phot_rp_n_obs,phot_rp_mean_flux,phot_rp_mean_flux_error,phot_rp_mean_mag
npartitions=3933,,,,,,,,,,,,,,,,,,,,,,,,,,,
"Order: 2, Pixel: 0",int64[pyarrow],string[pyarrow],int64[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],int64[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],int64[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],int64[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow]
"Order: 3, Pixel: 4",...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"Order: 4, Pixel: 3067",...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"Order: 3, Pixel: 767",...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [5]:
catwise = lsdb.open_catalog("/epyc/data3/hats/tmp/catwise/catwise2020")
catwise

,index,source_name,source_id,ra,dec,sigra,sigdec,sigradec,wx,wy,w1sky,w1sigsk,w1conf,w2sky,w2sigsk,w2conf,w1fitr,w2fitr,w1snr,w2snr,w1flux,w1sigflux,w2flux,w2sigflux,w1mpro,w1sigmpro,w1rchi2,w2mpro,w2sigmpro,w2rchi2,rchi2,nb,na,w1Sat,w2Sat,w1mag,w1sigm,w1flg,w1Cov,w2mag,w2sigm,w2flg,w2Cov,w1mag_1,w1sigm_1,w1flg_1,w2mag_1,w2sigm_1,w2flg_1,w1mag_2,w1sigm_2,w1flg_2,w2mag_2,w2sigm_2,w2flg_2,w1mag_3,w1sigm_3,w1flg_3,w2mag_3,w2sigm_3,w2flg_3,w1mag_4,w1sigm_4,w1flg_4,w2mag_4,w2sigm_4,w2flg_4,w1mag_5,w1sigm_5,w1flg_5,w2mag_5,w2sigm_5,w2flg_5,w1mag_6,w1sigm_6,w1flg_6,w2mag_6,w2sigm_6,w2flg_6,w1mag_7,w1sigm_7,w1flg_7,w2mag_7,w2sigm_7,w2flg_7,w1mag_8,w1sigm_8,w1flg_8,w2mag_8,w2sigm_8,w2flg_8,w1NM,w1M,w1magP,w1sigP1,w1sigP2,w1k,w1Ndf,w1mLQ,w1mJDmin,w1mJDmax,w1mJDmean,w2NM,w2M,w2magP,w2sigP1,w2sigP2,w2k,w2Ndf,w2mLQ,w2mJDmin,w2mJDmax,w2mJDmean,rho12,q12,nIters,nSteps,mdetID,p1,p2,MeanObsMJD,ra_pm,dec_pm,sigra_pm,sigdec_pm,sigradec_pm,PMRA,PMDec,sigPMRA,sigPMDec,w1snr_pm,w2snr_pm,w1flux_pm,w1sigflux_pm,w2flux_pm,w2sigflux_pm,w1mpro_pm,w1sigmpro_pm,w1rchi2_pm,w2mpro_pm,w2sigmpro_pm,w2rchi2_pm,rchi2_pm,pmcode,nIters_pm,nSteps_pm,dist,dw1mag,rch2w1,dw2mag,rch2w2,elon_avg,elonSig,elat_avg,elatSig,Delon,DelonSig,Delat,DelatSig,DelonSNR,DelatSNR,chi2pmra,chi2pmdec,ka,k1,k2,km,par_pm,par_pmSig,par_stat,par_sigma,dist_x,cc_flags,w1cc_map,w1cc_map_str,w2cc_map,w2cc_map_str,n_aw,ab_flags,w1ab_map,w1ab_map_str,w2ab_map,w2ab_map_str,glon,glat,elon,elat,unwise_objid
npartitions=4080,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
"Order: 4, Pixel: 0",int64[pyarrow],string[pyarrow],string[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],int64[pyarrow],int64[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],int64[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],int64[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],int64[pyarrow],double[pyarrow],double[pyarrow],int64[pyarrow],double[pyarrow],double[pyarrow],int64[pyarrow],double[pyarrow],double[pyarrow],int64[pyarrow],double[pyarrow],double[pyarrow],int64[pyarrow],double[pyarrow],double[pyarrow],int64[pyarrow],double[pyarrow],double[pyarrow],int64[pyarrow],double[pyarrow],double[pyarrow],int64[pyarrow],double[pyarrow],double[pyarrow],int64[pyarrow],double[pyarrow],double[pyarrow],int64[pyarrow],double[pyarrow],double[pyarrow],int64[pyarrow],double[pyarrow],double[pyarrow],int64[pyarrow],double[pyarrow],double[pyarrow],int64[pyarrow],double[pyarrow],double[pyarrow],int64[pyarrow],double[pyarrow],double[pyarrow],int64[pyarrow],double[pyarrow],double[pyarrow],int64[pyarrow],int64[pyarrow],int64[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],int64[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],int64[pyarrow],int64[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],int64[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],int64[pyarrow],int64[pyarrow],int64[pyarrow],int64[pyarrow],int64[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],string[pyarrow],int64[pyarrow],int64[pyarrow],double[py

## Joining using the Association table

The join is performed from gaia to catwise, using the `catalog.join` method, specifying the macauff association table to use to join through. This, like most LSDB operations, is also done lazily, and so the result is again a table schema with the gaia columns, the extra columns from the macauff table such as the probabilities and separations, and the wise columns.

To perform the computation, the `compute()` method is used to load the data and perform the join. In this case, we only compute a single partition so the result can be returned into memory. To run over the entire dataset and save to disk, the `write_catalog(path)` method can be used on the `joined` catalog.

In [6]:
joined = gaia.join(catwise, through=macauff_association, suffixes=("_gaia", "_catwise"))
joined

/astro/users/smcampos/.conda/envs/lsdb_v06/lib/python3.12/site-packages/lsdb/dask/join_catalog_data.py:393: RuntimeWarning: Right catalog does not have a margin cache. Results may be incomplete and/or inaccurate.
  warnings.warn(


,solution_id_gaia,designation_gaia,source_id_gaia,ref_epoch_gaia,ra_gaia,ra_error_gaia,dec_gaia,dec_error_gaia,parallax_gaia,parallax_error_gaia,pm_gaia,pmra_gaia,pmra_error_gaia,pmdec_gaia,pmdec_error_gaia,phot_g_n_obs_gaia,phot_g_mean_flux_gaia,phot_g_mean_flux_error_gaia,phot_g_mean_mag_gaia,phot_bp_n_obs_gaia,phot_bp_mean_flux_gaia,phot_bp_mean_flux_error_gaia,phot_bp_mean_mag_gaia,phot_rp_n_obs_gaia,phot_rp_mean_flux_gaia,phot_rp_mean_flux_error_gaia,phot_rp_mean_mag_gaia,gaia_ra,gaia_dec,BP,G,RP,catwise_ra,catwise_dec,W1,W2,match_p,separation,eta,xi,gaia_avg_cont,catwise_avg_cont,gaia_cont_f1,gaia_cont_f10,catwise_cont_f1,catwise_cont_f10,catwise_fit_sig,index_catwise,source_name_catwise,source_id_catwise,ra_catwise,dec_catwise,sigra_catwise,sigdec_catwise,sigradec_catwise,wx_catwise,wy_catwise,w1sky_catwise,w1sigsk_catwise,w1conf_catwise,w2sky_catwise,w2sigsk_catwise,w2conf_catwise,w1fitr_catwise,w2fitr_catwise,w1snr_catwise,w2snr_catwise,w1flux_catwise,w1sigflux_catwise,w2flux_catwise,w2sigflux_catwise,w1mpro_catwise,w1sigmpro_catwise,w1rchi2_catwise,w2mpro_catwise,w2sigmpro_catwise,w2rchi2_catwise,rchi2_catwise,nb_catwise,na_catwise,w1Sat_catwise,w2Sat_catwise,w1mag_catwise,w1sigm_catwise,w1flg_catwise,w1Cov_catwise,w2mag_catwise,w2sigm_catwise,w2flg_catwise,w2Cov_catwise,w1mag_1_catwise,w1sigm_1_catwise,w1flg_1_catwise,w2mag_1_catwise,w2sigm_1_catwise,w2flg_1_catwise,w1mag_2_catwise,w1sigm_2_catwise,w1flg_2_catwise,w2mag_2_catwise,w2sigm_2_catwise,w2flg_2_catwise,w1mag_3_catwise,w1sigm_3_catwise,w1flg_3_catwise,w2mag_3_catwise,w2sigm_3_catwise,w2flg_3_catwise,w1mag_4_catwise,w1sigm_4_catwise,w1flg_4_catwise,w2mag_4_catwise,w2sigm_4_catwise,w2flg_4_catwise,w1mag_5_catwise,w1sigm_5_catwise,w1flg_5_catwise,w2mag_5_catwise,w2sigm_5_catwise,w2flg_5_catwise,w1mag_6_catwise,w1sigm_6_catwise,w1flg_6_catwise,w2mag_6_catwise,w2sigm_6_catwise,w2flg_6_catwise,w1mag_7_catwise,w1sigm_7_catwise,w1flg_7_catwise,w2mag_7_catwise,w2sigm_7_catwise,w2flg_7_catwise,w1mag_8_catwise,w1sigm_8_catwise,w1flg_8_catwise,w2mag_8_catwise,w2sigm_8_catwise,w2flg_8_catwise,w1NM_catwise,w1M_catwise,w1magP_catwise,w1sigP1_catwise,w1sigP2_catwise,w1k_catwise,w1Ndf_catwise,w1mLQ_catwise,w1mJDmin_catwise,w1mJDmax_catwise,w1mJDmean_catwise,w2NM_catwise,w2M_catwise,w2magP_catwise,w2sigP1_catwise,w2sigP2_catwise,w2k_catwise,w2Ndf_catwise,w2mLQ_catwise,w2mJDmin_catwise,w2mJDmax_catwise,w2mJDmean_catwise,rho12_catwise,q12_catwise,nIters_catwise,nSteps_catwise,mdetID_catwise,p1_catwise,p2_catwise,MeanObsMJD_catwise,ra_pm_catwise,dec_pm_catwise,sigra_pm_catwise,sigdec_pm_catwise,sigradec_pm_catwise,PMRA_catwise,PMDec_catwise,sigPMRA_catwise,sigPMDec_catwise,w1snr_pm_catwise,w2snr_pm_catwise,w1flux_pm_catwise,w1sigflux_pm_catwise,w2flux_pm_catwise,w2sigflux_pm_catwise,w1mpro_pm_catwise,w1sigmpro_pm_catwise,w1rchi2_pm_catwise,w2mpro_pm_catwise,w2sigmpro_pm_catwise,w2rchi2_pm_catwise,rchi2_pm_catwise,pmcode_catwise,nIters_pm_catwise,nSteps_pm_catwise,dist_catwise,dw1mag_catwise,rch2w1_catwise,dw2mag_catwise,rch2w2_catwise,elon_avg_catwise,elonSig_catwise,elat_avg_catwise,elatSig_catwise,Delon_catwise,DelonSig_catwise,Delat_catwise,DelatSig_catwise,DelonSNR_catwise,DelatSNR_catwise,chi2pmra_catwise,chi2pmdec_catwise,ka_catwise,k1_catwise,k2_catwise,km_catwise,par_pm_catwise,par_pmSig_catwise,par_stat_catwise,par_sigma_catwise,dist_x_catwise,cc_flags_catwise,w1cc_map_catwise,w1cc_map_str_catwise,w2cc_map_catwise,w2cc_map_str_catwise,n_aw_catwise,ab_flags_catwise,w1ab_map_catwise,w1ab_map_str_catwise,w2ab_map_catwise,w2ab_map_str_catwise,glon_catwise,glat_catwise,elon_catwise,elat_catwise,unwise_objid_catwise
npartitions=5619,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
"Order: 4, Pixel: 0",int64[pyarrow],string[pyarrow],int64[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow

In [7]:
joined._ddf.partitions[0].compute()

,solution_id_gaia,designation_gaia,source_id_gaia,ref_epoch_gaia,ra_gaia,ra_error_gaia,dec_gaia,dec_error_gaia,parallax_gaia,parallax_error_gaia,pm_gaia,pmra_gaia,pmra_error_gaia,pmdec_gaia,pmdec_error_gaia,phot_g_n_obs_gaia,phot_g_mean_flux_gaia,phot_g_mean_flux_error_gaia,phot_g_mean_mag_gaia,phot_bp_n_obs_gaia,phot_bp_mean_flux_gaia,phot_bp_mean_flux_error_gaia,phot_bp_mean_mag_gaia,phot_rp_n_obs_gaia,phot_rp_mean_flux_gaia,phot_rp_mean_flux_error_gaia,phot_rp_mean_mag_gaia,gaia_ra,gaia_dec,BP,G,RP,catwise_ra,catwise_dec,W1,W2,match_p,separation,eta,xi,gaia_avg_cont,catwise_avg_cont,gaia_cont_f1,gaia_cont_f10,catwise_cont_f1,catwise_cont_f10,catwise_fit_sig,index_catwise,source_name_catwise,source_id_catwise,ra_catwise,dec_catwise,sigra_catwise,sigdec_catwise,sigradec_catwise,wx_catwise,wy_catwise,w1sky_catwise,w1sigsk_catwise,w1conf_catwise,w2sky_catwise,w2sigsk_catwise,w2conf_catwise,w1fitr_catwise,w2fitr_catwise,w1snr_catwise,w2snr_catwise,w1flux_catwise,w1sigflux_catwise,w2flux_catwise,w2sigflux_catwise,w1mpro_catwise,w1sigmpro_catwise,w1rchi2_catwise,w2mpro_catwise,w2sigmpro_catwise,w2rchi2_catwise,rchi2_catwise,nb_catwise,na_catwise,w1Sat_catwise,w2Sat_catwise,w1mag_catwise,w1sigm_catwise,w1flg_catwise,w1Cov_catwise,w2mag_catwise,w2sigm_catwise,w2flg_catwise,w2Cov_catwise,w1mag_1_catwise,w1sigm_1_catwise,w1flg_1_catwise,w2mag_1_catwise,w2sigm_1_catwise,w2flg_1_catwise,w1mag_2_catwise,w1sigm_2_catwise,w1flg_2_catwise,w2mag_2_catwise,w2sigm_2_catwise,w2flg_2_catwise,w1mag_3_catwise,w1sigm_3_catwise,w1flg_3_catwise,w2mag_3_catwise,w2sigm_3_catwise,w2flg_3_catwise,w1mag_4_catwise,w1sigm_4_catwise,w1flg_4_catwise,w2mag_4_catwise,w2sigm_4_catwise,w2flg_4_catwise,w1mag_5_catwise,w1sigm_5_catwise,w1flg_5_catwise,w2mag_5_catwise,w2sigm_5_catwise,w2flg_5_catwise,w1mag_6_catwise,w1sigm_6_catwise,w1flg_6_catwise,w2mag_6_catwise,w2sigm_6_catwise,w2flg_6_catwise,w1mag_7_catwise,w1sigm_7_catwise,w1flg_7_catwise,w2mag_7_catwise,w2sigm_7_catwise,w2flg_7_catwise,w1mag_8_catwise,w1sigm_8_catwise,w1flg_8_catwise,w2mag_8_catwise,w2sigm_8_catwise,w2flg_8_catwise,w1NM_catwise,w1M_catwise,w1magP_catwise,w1sigP1_catwise,w1sigP2_catwise,w1k_catwise,w1Ndf_catwise,w1mLQ_catwise,w1mJDmin_catwise,w1mJDmax_catwise,w1mJDmean_catwise,w2NM_catwise,w2M_catwise,w2magP_catwise,w2sigP1_catwise,w2sigP2_catwise,w2k_catwise,w2Ndf_catwise,w2mLQ_catwise,w2mJDmin_catwise,w2mJDmax_catwise,w2mJDmean_catwise,rho12_catwise,q12_catwise,nIters_catwise,nSteps_catwise,mdetID_catwise,p1_catwise,p2_catwise,MeanObsMJD_catwise,ra_pm_catwise,dec_pm_catwise,sigra_pm_catwise,sigdec_pm_catwise,sigradec_pm_catwise,PMRA_catwise,PMDec_catwise,sigPMRA_catwise,sigPMDec_catwise,w1snr_pm_catwise,w2snr_pm_catwise,w1flux_pm_catwise,w1sigflux_pm_catwise,w2flux_pm_catwise,w2sigflux_pm_catwise,w1mpro_pm_catwise,w1sigmpro_pm_catwise,w1rchi2_pm_catwise,w2mpro_pm_catwise,w2sigmpro_pm_catwise,w2rchi2_pm_catwise,rchi2_pm_catwise,pmcode_catwise,nIters_pm_catwise,nSteps_pm_catwise,dist_catwise,dw1mag_catwise,rch2w1_catwise,dw2mag_catwise,rch2w2_catwise,elon_avg_catwise,elonSig_catwise,elat_avg_catwise,elatSig_catwise,Delon_catwise,DelonSig_catwise,Delat_catwise,DelatSig_catwise,DelonSNR_catwise,DelatSNR_catwise,chi2pmra_catwise,chi2pmdec_catwise,ka_catwise,k1_catwise,k2_catwise,km_catwise,par_pm_catwise,par_pmSig_catwise,par_stat_catwise,par_sigma_catwise,dist_x_catwise,cc_flags_catwise,w1cc_map_catwise,w1cc_map_str_catwise,w2cc_map_catwise,w2cc_map_str_catwise,n_aw_catwise,ab_flags_catwise,w1ab_map_catwise,w1ab_map_str_catwise,w2ab_map_catwise,w2ab_map_str_catwise,glon_catwise,glat_catwise,elon_catwise,elat_catwise,unwise_objid_catwise
_healpix_29,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
3121269471,1636148068921376768,Gaia DR3 4295806720,4295806720,2016.0,44.996155,0.101618,0.005615,0.101334,0.354331,0.1

## Querying the joined table

The catalog's `query()` method can be used to perform queries on the data. The query strings follow the pandas query function, for which the [full documentation is available here](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.query.html).

In this case, we query on the `match_p` column, getting rows with values over `0.999`.

In [8]:
joined.query("match_p > 0.999")._ddf.partitions[0].compute()

,solution_id_gaia,designation_gaia,source_id_gaia,ref_epoch_gaia,ra_gaia,ra_error_gaia,dec_gaia,dec_error_gaia,parallax_gaia,parallax_error_gaia,pm_gaia,pmra_gaia,pmra_error_gaia,pmdec_gaia,pmdec_error_gaia,phot_g_n_obs_gaia,phot_g_mean_flux_gaia,phot_g_mean_flux_error_gaia,phot_g_mean_mag_gaia,phot_bp_n_obs_gaia,phot_bp_mean_flux_gaia,phot_bp_mean_flux_error_gaia,phot_bp_mean_mag_gaia,phot_rp_n_obs_gaia,phot_rp_mean_flux_gaia,phot_rp_mean_flux_error_gaia,phot_rp_mean_mag_gaia,gaia_ra,gaia_dec,BP,G,RP,catwise_ra,catwise_dec,W1,W2,match_p,separation,eta,xi,gaia_avg_cont,catwise_avg_cont,gaia_cont_f1,gaia_cont_f10,catwise_cont_f1,catwise_cont_f10,catwise_fit_sig,index_catwise,source_name_catwise,source_id_catwise,ra_catwise,dec_catwise,sigra_catwise,sigdec_catwise,sigradec_catwise,wx_catwise,wy_catwise,w1sky_catwise,w1sigsk_catwise,w1conf_catwise,w2sky_catwise,w2sigsk_catwise,w2conf_catwise,w1fitr_catwise,w2fitr_catwise,w1snr_catwise,w2snr_catwise,w1flux_catwise,w1sigflux_catwise,w2flux_catwise,w2sigflux_catwise,w1mpro_catwise,w1sigmpro_catwise,w1rchi2_catwise,w2mpro_catwise,w2sigmpro_catwise,w2rchi2_catwise,rchi2_catwise,nb_catwise,na_catwise,w1Sat_catwise,w2Sat_catwise,w1mag_catwise,w1sigm_catwise,w1flg_catwise,w1Cov_catwise,w2mag_catwise,w2sigm_catwise,w2flg_catwise,w2Cov_catwise,w1mag_1_catwise,w1sigm_1_catwise,w1flg_1_catwise,w2mag_1_catwise,w2sigm_1_catwise,w2flg_1_catwise,w1mag_2_catwise,w1sigm_2_catwise,w1flg_2_catwise,w2mag_2_catwise,w2sigm_2_catwise,w2flg_2_catwise,w1mag_3_catwise,w1sigm_3_catwise,w1flg_3_catwise,w2mag_3_catwise,w2sigm_3_catwise,w2flg_3_catwise,w1mag_4_catwise,w1sigm_4_catwise,w1flg_4_catwise,w2mag_4_catwise,w2sigm_4_catwise,w2flg_4_catwise,w1mag_5_catwise,w1sigm_5_catwise,w1flg_5_catwise,w2mag_5_catwise,w2sigm_5_catwise,w2flg_5_catwise,w1mag_6_catwise,w1sigm_6_catwise,w1flg_6_catwise,w2mag_6_catwise,w2sigm_6_catwise,w2flg_6_catwise,w1mag_7_catwise,w1sigm_7_catwise,w1flg_7_catwise,w2mag_7_catwise,w2sigm_7_catwise,w2flg_7_catwise,w1mag_8_catwise,w1sigm_8_catwise,w1flg_8_catwise,w2mag_8_catwise,w2sigm_8_catwise,w2flg_8_catwise,w1NM_catwise,w1M_catwise,w1magP_catwise,w1sigP1_catwise,w1sigP2_catwise,w1k_catwise,w1Ndf_catwise,w1mLQ_catwise,w1mJDmin_catwise,w1mJDmax_catwise,w1mJDmean_catwise,w2NM_catwise,w2M_catwise,w2magP_catwise,w2sigP1_catwise,w2sigP2_catwise,w2k_catwise,w2Ndf_catwise,w2mLQ_catwise,w2mJDmin_catwise,w2mJDmax_catwise,w2mJDmean_catwise,rho12_catwise,q12_catwise,nIters_catwise,nSteps_catwise,mdetID_catwise,p1_catwise,p2_catwise,MeanObsMJD_catwise,ra_pm_catwise,dec_pm_catwise,sigra_pm_catwise,sigdec_pm_catwise,sigradec_pm_catwise,PMRA_catwise,PMDec_catwise,sigPMRA_catwise,sigPMDec_catwise,w1snr_pm_catwise,w2snr_pm_catwise,w1flux_pm_catwise,w1sigflux_pm_catwise,w2flux_pm_catwise,w2sigflux_pm_catwise,w1mpro_pm_catwise,w1sigmpro_pm_catwise,w1rchi2_pm_catwise,w2mpro_pm_catwise,w2sigmpro_pm_catwise,w2rchi2_pm_catwise,rchi2_pm_catwise,pmcode_catwise,nIters_pm_catwise,nSteps_pm_catwise,dist_catwise,dw1mag_catwise,rch2w1_catwise,dw2mag_catwise,rch2w2_catwise,elon_avg_catwise,elonSig_catwise,elat_avg_catwise,elatSig_catwise,Delon_catwise,DelonSig_catwise,Delat_catwise,DelatSig_catwise,DelonSNR_catwise,DelatSNR_catwise,chi2pmra_catwise,chi2pmdec_catwise,ka_catwise,k1_catwise,k2_catwise,km_catwise,par_pm_catwise,par_pmSig_catwise,par_stat_catwise,par_sigma_catwise,dist_x_catwise,cc_flags_catwise,w1cc_map_catwise,w1cc_map_str_catwise,w2cc_map_catwise,w2cc_map_str_catwise,n_aw_catwise,ab_flags_catwise,w1ab_map_catwise,w1ab_map_str_catwise,w2ab_map_catwise,w2ab_map_str_catwise,glon_catwise,glat_catwise,elon_catwise,elat_catwise,unwise_objid_catwise
_healpix_29,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
3121269471,1636148068921376768,Gaia DR3 4295806720,4295806720,2016.0,44.996155,0.101618,0.005615,0.101334,0.354331,0.1

In [10]:
client.close()